In [ ]:
! git clone https://github.com/intel/e2eAIOK.git /tmp/e2eAIOK

In [ ]:
! apt-get update -y &&  DEBIAN_FRONTEND=noninteractive apt-get install -y python3 python3-pip python-is-python3 graphviz

In [ ]:
! DEBIAN_FRONTEND=noninteractive apt-get install -y openjdk-8-jre

In [ ]:
! cd /tmp/e2eAIOK/RecDP/ && python setup.py sdist && pip install dist/pyrecdp-*.tar.gz

In [1]:
import os
from pyrecdp.primitives.llmutils import classify_spark, language_identify_spark
from pyrecdp.primitives.llmutils.utils import get_target_file_list
from huggingface_hub import hf_hub_download
from pyrecdp.core import SparkDataProcessor

JAVA_HOME is not set, use default value of /usr/lib/jvm/java-8-openjdk-amd64/


/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
fasttext_model_dir = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
data_dir = "/tmp/e2eAIOK/RecDP/tests/data/llm_data"
data_file = "/tmp/e2eAIOK/RecDP/tests/data/llm_data/arxiv_sample_100.jsonl"
classify_save_path = "/tmp/e2eAIOK/RecDP/tests/data/output/classify_spark"
lid_save_path = "/tmp/e2eAIOK/RecDP/tests/data//output/lid_spark"

rdp = SparkDataProcessor()
spark=rdp.spark
spark_df = spark.read.json(data_file)
lid_df = language_identify_spark(spark_df, fasttext_model_dir, 'text', 'lang', lid_save_path)
print("input is")
lid_df.show()
classify_spark(lid_df, "lang", classify_save_path, "file://")

Will assign 1 cores and 10386 M memory for spark
per core memory size is 10.143 GB and shuffle_disk maximum capacity is 8589934592.000 GB
process data started ...
process data took 0.6920480530006898 sec
Save data started ...
Save data took 30.996972035 sec
Completed!!
    total identify the language for 100 documents
    All the processed data are saving under the folder: /tmp/e2eAIOK/RecDP/tests/data//output/lid_spark
input is
+--------------------+--------------------+--------+
|                meta|                text|    lang|
+--------------------+--------------------+--------+
|{2203.15369, en, ...|\section{Introduc...|eng_Latn|
|{math/9807097, en...|\section{Introduc...|eng_Latn|
|{2008.06948, en, ...|\section{Introduc...|eng_Latn|
|{cond-mat/9807071...|\section{Introduc...|eng_Latn|
|{2210.10650, en, ...|\section{\label{s...|eng_Latn|
|{astro-ph/9807119...|\section{Introduc...|eng_Latn|
|{2111.03152, en, ...|\section{Introduc...|eng_Latn|
|{1606.04992, en, ...|\n\n\section{In

DataFrame[meta: struct<arxiv_id:string,language:string,timestamp:string,url:string,yymm:string>, text: string, lang: string]

In [4]:
! ls $classify_save_path

'lang=eng_Latn'   _SUCCESS
